<a href="https://colab.research.google.com/github/eugeniaaaaaaaaaaaa/linear_regression/blob/main/%D0%98%D1%82%D0%BE%D0%B3%D0%BE%D0%B2%D1%8B%D0%B9_%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82_DA%26ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Кейс «Анализ факторов, влияющих на употребление табака в США»
**Аннотация.** Курение, несмотря на все существующие предупреждения о его вредности, продолжает оставаться одной из самых распространенных вредных привычек среди населения США. Каждый год тысячи людей умирают от заболеваний, связанных с курением. Изучение причин курения и поиски эффективных способов борьбы с этой привычкой являются важными задачами в области общественного здравоохранения. Программы по борьбе с курением, включающие информационные кампании, консультации и другие меры, могут помочь людям бросить курить и избежать многих заболеваний, связанных с курением. Поэтому изучение данных об употреблении табака и поиск способов борьбы с курением являются важными шагами на пути к более здоровому обществу.

**Цель:** проанализировать данные по употреблению табака в США. Выявить основные факторы, влияющие на уровень употребления табака и предложить прогноз (построить регрессионную модель) по уровню употреблению табака в США в зависимости от выявленных факторов.

**Техническое задание:** требуется проанализировать данные по употреблению табака в США и зависимости количества людей, курящих ежедневно, от таких факторов как год, штат, процент бросивших курить, процент никогда не курящих и процент людей, иногда употребляющих табак, а также дать прогноз по употреблению табака в США. Построить качественную модель и объяснить ее выбор. По построенной модели дать интерпретацию (какие факторы влияют на вероятность курения или бросания курения и определить наиболее важные факторы, которые можно использовать для разработки эффективных программ по борьбе с курением) и дать прогноз по будущим тенденциям и изменениям употребления табака в США. Пояснить полученные результаты.


В датасете есть полные данные (16 - c 1995 по 2010) по 50 штатам, District of Columbia - 15 (с 1996), Puerto Rico - 15(с 1996), Hawaii - 15(без 2004), Utah - 14(с 1997), Virgin Islands - 10 (с 2001), Guam - 7 (с 2001 кроме 2004). Пуэрто-Рико,  Виргинские Острова, Гуам - островные территории.

Еще есть общие полные данные - Nationwide (States, DC, and Territories) и Nationwide (States and DC)

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.stats.api as sms
import statsmodels.api as sm
import sklearn.metrics as metrics
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
def chaddock(_corrcoef):
  _corr = abs(_corrcoef)
  _vals = ((0.1, 'отсутствует'), (0.3, 'слабая'), (0.5, 'умеренная'),
           (0.7, 'заметная'), (0.9, 'высокая'), (1, 'очень высокая'))
  _ans = 'коэффициент корреляции вычислен неверно'
  for _v in _vals:
    if _corr <= _v[0]:
      _ans = _v[1]
      break
  return _ans

def r2_adj(X, y, y_count):
  n = len(y)  # объем выборки, на которой строили модель
  k = len(X) + 1  # количество параметров модели
  return 1 - (1 - metrics.r2_score(y, y_count)) * (n - 1) / (n - k)

In [5]:
df = pd.read_csv('https://github.com/eugeniaaaaaaaaaaaa/linear_regression/blob/main/Smoking.csv', sep=';')
df = df.drop('index', axis=1)

df.columns = ['year', 'state', 'everyday', 'sometimes', 'former', 'never']
smokers_data = ['everyday', 'sometimes', 'former', 'never']
names = {
    'year': 'Год',
    'state': 'Название штата',
    'everyday': 'Курящие каждый день',
    'sometimes': 'Курящие иногда',
    'former': 'Бросившие курить',
    'never': 'Никогда не курившие'
}

print(f'Количество пропусков в данных:\n{df.isna().sum()}')

df[smokers_data] = df[smokers_data].applymap(lambda v: float(v[:-1]) / 100)
df_common = df[df.state == 'Nationwide (States, DC, and Territories)']
df_no_ter =  df[df.state =='Nationwide (States and DC)']
df = df[~((df.state == 'Nationwide (States, DC, and Territories)') | (df.state =='Nationwide (States and DC)'))]#.sort_values(by=['Year'], ascending=True)

print('''
Пропусков в строках данных нет, однако в датасете отсутствуют следующие данные:
- District of Columbia (1995)
- Puerto Rico (1995)
- Hawaii (2004)
- Utah (1995-1996)
- Virgin Islands (1995-2000)
- Guam (1995-2000, 2004)
''')

print(f'''
Проверка корректности данных - суммируем доли опрошенных
Основной датасет: {df[smokers_data].sum(axis=1).mean():.3f}
Nationwide (States, DC, and Territories): {df_common[smokers_data].sum(axis=1).mean():.3f}
Nationwide (States and DC): {df_no_ter[smokers_data].sum(axis=1).mean():.3f}
0.7% вероятно потеряны из-за округления ''')

ParserError: Error tokenizing data. C error: Expected 1 fields in line 35, saw 2


In [ ]:
# переменную State превратим в дамми-переменные
dummy = pd.get_dummies(
    df.state,
    drop_first=False
)
df = pd.concat([df, dummy], axis="columns")
df.sample(5, random_state=7)

In [ ]:
print('Среднее:')
print(df[smokers_data].mean())

print('\nМода:')
print(df[smokers_data].mode())

print('\nМедиана:')
print(df[smokers_data].median())

print('\nСтандартное отклонение:')
print(df[smokers_data].std())


print('\nКвантили:')
print(pd.concat([df[smokers_data].quantile(q=0.25), df[smokers_data].quantile(q=0.5), df[smokers_data].quantile(q=0.75)], axis=1, keys=['Q1', 'Q2', 'Q3']))

In [ ]:
df[smokers_data].describe()[1:]

In [ ]:
# проверка на нормальность
plt.rcParams.update({'font.size': 20})
plt.figure(figsize=(15, 3))

plt.subplot(1, 2, 1)
plt.hist(df.year, bins=10, color="lightblue", linewidth=1, edgecolor="black")
plt.xlabel(names["year"] )
plt.ylabel("Частота")

plt.subplot(1, 2, 2)
plt.hist(df.state, bins=len(df.state.unique()), color="lightblue", linewidth=1, edgecolor="black", label='enabled')
plt.xlabel(names["state"])
plt.ylabel("Частота")
plt.xticks([])

plt.show()


plt.figure(figsize=(28, 5))

plt.subplot(1, 4, 1)
plt.hist(df[smokers_data[0]], bins=10, color="coral", linewidth=1, edgecolor="black")
plt.xlabel(names[smokers_data[0]])
plt.ylabel("Частота")
for i in range(1, 4):
  plt.subplot(1, 4, i + 1)
  plt.hist(df[smokers_data[i]], bins=10, color="coral", linewidth=1, edgecolor="black")
  plt.xlabel(names[smokers_data[i]])


In [ ]:
alpha = 0.05
isnorm = lambda p_value: ('ненормально', 'нормально')[int(p_value >= alpha)]
print(f'''Проверим данные на нормальность с помощью критерия Жарка-Бера.
Уровень значимости α = 0.05\n''')
for col in smokers_data + ['year']:
  p_val = stats.jarque_bera(df[col])[1]
  print(f'{names[col]}:  p = {p_val:.3}, распределение {isnorm(p_val)}')

In [ ]:
# проверим данные на упорядоченность
# используем критерий Вальда-Волфовитца
import statsmodels
print("p-значение:", statsmodels.sandbox.stats.runs.runstest_1samp(df.everyday)[1])
print("p-значение после перемешивания:", statsmodels.sandbox.stats.runs.runstest_1samp(df.sample(len(df), random_state=6786866).everyday)[1])
df = df.sample(len(df), random_state=6786866)

In [ ]:
corr_matrix = df.corr(method="spearman", numeric_only=True)

In [ ]:
plt.figure(figsize=(50, 5))

sns.heatmap(corr_matrix[:5], annot=True, fmt=".2f")

plt.show()

In [ ]:
X_cols = [col for col in corr_matrix.columns if abs(corr_matrix[col]['everyday']) > 0.1] # хотя бы слабая корреляция с everyday
corr_matrix = df[X_cols].corr(method="spearman")


plt.figure(figsize=(9, 8))

sns.heatmap(corr_matrix[:], annot=True, fmt=".1f")

plt.show()


X_cols = corr_matrix['everyday'].sort_values(ascending=False, key=abs).keys()[1:]

In [ ]:
# линейная модель со всеми факторами

y_col = 'everyday'
X_train, X_test, y_train, y_test = train_test_split(df[X_cols], df[y_col],
                                                    test_size=0.2, random_state=999)

model = LinearRegression()
model.fit(X=X_train, y=y_train)
# print(f"Коэффициенты модели: {model.coef_}, свободный член: {model.intercept_}")

y_model = model.predict(X_train)
r2_train= metrics.r2_score(y_train, y_model)

y_pred = model.predict(X_test)
r2_test = metrics.r2_score(y_test, y_pred)
print(f"R2_train = {round(r2_train, 3)}, R2_test = {round(r2_test, 3)}")

r2_adj_train, r2_adj_test = r2_adj(X_cols, y_train, y_model), r2_adj(X_cols, y_test, y_pred)
print(f"R2_adj_train = {round(r2_adj_train, 3)}, R2_adj_test = {round(r2_adj_test, 3)}")

print('Модель', ('не переобучена', 'переобучена')[int(r2_adj_train - r2_adj_test > 0.1)])

# вычислим метрики MAE и RMSE на тестовой выборке
MAE = metrics.mean_absolute_error(y_test, y_pred)
RMSE = metrics.mean_squared_error(y_test, y_pred) ** 0.5

print(f"MAE = {round(MAE, 3)}, RMSE = {round(RMSE, 3)}")
print('\n\nКоэффициенты при целевых признаках')
print(*list(zip(X_cols, model.coef_)), sep='\n')

# проверка теоремы Гаусса-Маркова
resid = y_train - y_model
exog = X_train.copy()
exog['const'] = np.ones(len(X_train))

alpha = 0.05
p_mist = stats.ttest_1samp(a=resid, popmean=0)[1]
p_homo = sm.stats.diagnostic.het_breuschpagan(resid, exog)[1]
dw = sm.stats.stattools.durbin_watson(resid)
p_jb = stats.jarque_bera(resid)[1]
print(f'''\n\n
1) Значения факторных признаков не случайны - выполняется.
2) Математическое ожидание ошибки модели равно 0: p={p_mist:.3f} - {('выполняется', 'не выполняется')[int(p_mist < alpha)]}.
3) Ошибки модели гомоскедастичны (их дисперсия постоянна): p={p_homo:.3f} - {('выполняется', 'не выполняется')[int(p_mist < alpha)]}.
4) Ошибки модели должны быть некоррелированы (независимы): DW={dw:.3f} - {('не выполняется', 'выполняется')[int(1.5 < dw < 2.5)]}.
5) Ошибки модели должны иметь нормальное распределение: p={p_jb:.3f} - {('выполняется', 'не выполняется')[int(p_mist < alpha)]}.
''')


С каждым годом доля курящих ежедневно уменьшается на 3.7%, а при увеличении количества никогда не куривших людей на 1% доля курящих ежедневно уменьшается на 0.27. Кроме того доля ежедневно курящих меняется в зависимости от штата.

In [ ]:
X_cols1 = [col for col in corr_matrix.columns if abs(corr_matrix[col]['everyday']) > 0.3] # умеренная корреляция с everyday
corr_matrix1 = df[X_cols1].corr(method="spearman")
plt.figure(figsize=(4, 3))
sns.heatmap(corr_matrix1[:], annot=True, fmt=".1f")
# hmap = sns.heatmap(df.corr(numeric_only=True), annot=True, fmt=".1f")
plt.show()

X_cols1 = corr_matrix1['everyday'].sort_values(ascending=False, key=abs).keys()[1:]# линейная модель

y_col = 'everyday'
X_train1, X_test1, y_train, y_test = train_test_split(
  df[X_cols1], df[y_col], test_size=0.2, random_state=999
)

model1 = LinearRegression()
model1.fit(X=X_train1, y=y_train)
# print(f"Коэффициенты модели: {model.coef_}, свободный член: {model.intercept_}")

y_model1 = model1.predict(X_train1)
r2_train1= metrics.r2_score(y_train, y_model1)

y_pred1 = model1.predict(X_test1)
r2_test1 = metrics.r2_score(y_test, y_pred1)
print(f"R2_train = {round(r2_train1, 3)}, R2_test = {round(r2_test1, 3)}")

r2_adj_train1, r2_adj_test1 = r2_adj(X_cols1, y_train, y_model1), r2_adj(X_cols1, y_test, y_pred1)
print(f"R2_adj_train = {round(r2_adj_train1, 3)}, R2_adj_test = {round(r2_adj_test1, 3)}")

print('Модель', ('не переобучена', 'переобучена')[int(r2_adj_train1 - r2_adj_test1 > 0.1)])

# вычислим метрики MAE и RMSE на тестовой выборке
MAE1 = metrics.mean_absolute_error(y_test, y_pred1)
# в версии sklearn в Colab отсутствует готовая функция для RMSE
RMSE1 = metrics.mean_squared_error(y_test, y_pred1) ** 0.5

print(f"MAE = {round(MAE1, 3)}, RMSE = {round(RMSE1, 3)}")
print('\n\nКоэффициенты при целевых признаках')
print(*list(zip(X_cols1, model1.coef_)), sep='\n')

# проверка теоремы Гаусса-Маркова
import statsmodels.stats.api as sms
resid1= y_train - y_model1
exog1 = X_train1.copy()
exog1['const'] = np.ones(len(X_train1))

alpha = 0.05
p_mist1 = stats.ttest_1samp(a=resid1, popmean=0)[1]
p_homo1 = sm.stats.diagnostic.het_breuschpagan(resid1, exog1)[1]
dw1 = sm.stats.stattools.durbin_watson(resid1)
p_jb1 = stats.jarque_bera(resid1)[1]
print(f'''\n\n
1) Значения факторных признаков не случайны - выполняется.
2) Математическое ожидание ошибки модели равно 0: p={p_mist1:.3f} - {('выполняется', 'не выполняется')[int(p_mist1 < alpha)]}.
3) Ошибки модели гомоскедастичны (их дисперсия постоянна): p={p_homo1:.3f} - {('выполняется', 'не выполняется')[int(p_mist1 < alpha)]}.
4) Ошибки модели должны быть некоррелированы (независимы): DW={dw1:.3f} - {('не выполняется', 'выполняется')[int(1.5 < dw1 < 2.5)]}.
5) Ошибки модели должны иметь нормальное распределение: p={p_jb1:.3f} - {('выполняется', 'не выполняется')[int(p_mist1 < alpha)]}.
''')